In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
data = pd.read_csv('clean_data.csv')

# Separate features and labels
X = data[['left_shoulder_y', 'left_shoulder_x', 'right_shoulder_y', 'right_shoulder_x', 'left_body_y', 'left_body_x', 'right_body_y', 'right_body_x', 'len_factor', 'left_knee_y', 'left_knee_x', 'right_knee_y', 'right_knee_x', 'left_foot_y', 'right_foot_y']]
y = data['fall'].astype(int)  # Convert boolean to int (1 for True, 0 for False)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Generated {len(X_train)} training samples and {len(X_test)} testing samples")

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Generated 168 training samples and 43 testing samples


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from kerastuner import HyperModel, RandomSearch

class MyHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=512, step=32),
                        activation='relu', input_shape=(X_train.shape[1],)))
        
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(Dense(units=hp.Int(f'units_{i+2}', min_value=32, max_value=512, step=32),
                            activation='relu'))
        
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        return model

tuner = RandomSearch(
    MyHyperModel(),
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=2,
    directory='keras_tuner',
    project_name='fall_detection'
)

tuner.search(X_train, y_train, epochs=100, validation_split=0.2)
best_model = tuner.get_best_models(num_models=1)[0]
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential({
    Dense(160, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(320, activation='relu'),
    Dense(192, activation='relu'),
    Dense(1, activation='sigmoid')
})
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

TypeError: The added layer must be an instance of class Layer. Received: layer={<keras.src.layers.core.dense.Dense object at 0x164002510>, <keras.src.layers.core.dense.Dense object at 0x164002290>, <keras.src.layers.core.dense.Dense object at 0x164934610>, <keras.src.layers.core.dense.Dense object at 0x164944f50>} of type <class 'set'>.

# BEST MODEL
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 dense (Dense)               (None, 160)               2560      
                                                                 
 dense_1 (Dense)             (None, 320)               51520     
                                                                 
 dense_2 (Dense)             (None, 192)               61632     
                                                                 
 dense_3 (Dense)             (None, 1)                 193       
                                                                 
=================================================================
Total params: 115905 (452.75 KB)
Trainable params: 115905 (452.75 KB)
Non-trainable params: 0 (0.00 Byte)

In [ ]:
history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_split=0.2)# Evaluate the model
loss, accuracy = model..evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Test the model
print(f"Predicting {len(X_test)} test samples")
y_pred = best_model.predict(X_test)
y_pred = np.round(y_pred)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted No Fall', 'Predicted Fall'], yticklabels=['Actual No Fall', 'Actual Fall'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matfrix')
plt.show()

# Optionally, print the classification report for more detailed metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))